<a href="https://colab.research.google.com/github/bkk443/IgGM_mag/blob/main/RFantibodytest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 5.3 MB/s eta 0:00:00


In [2]:
# Revised Step 2: Install Docker and NVIDIA tools without systemd
!curl -fsSL https://get.docker.com -o get-docker.sh
!sh get-docker.sh
!mkdir -p /etc/docker
!echo '{"exec-opts": ["native.cgroupdriver=cgroupfs"]}' > /etc/docker/daemon.json
!apt-get install -y nvidia-container-toolkit-base
!nvidia-ctk runtime configure --runtime=docker
!pkill -SIGHUP dockerd  # Reload docker daemon without systemd

# Executing docker install script, commit: 4c94a56999e10efcf48c5b8e3f6afea464f9108e
+ sh -c apt-get -qq update >/dev/null
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu jammy stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sh -c DEBIAN_

In [3]:
# Step 3: Clone repository and prepare environment
!git clone https://github.com/RosettaCommons/RFantibody.git
%cd RFantibody

Cloning into 'RFantibody'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 230 (delta 40), reused 230 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (230/230), 1.94 MiB | 23.37 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/RFantibody


In [4]:
# Step 4: Download weights
!bash include/download_weights.sh


--2025-03-02 16:41:03--  https://files.ipd.uw.edu/pub/RFantibody/RFdiffusion_Ab.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.134, 128.95.160.135, 2607:4000:406::160:135, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 483452922 (461M) [application/octet-stream]
Saving to: ‘RFdiffusion_Ab.pt’

RFdiffusion_Ab.pt   100%[===================>] 461.06M  8.90MB/s    in 46s     

2025-03-02 16:41:51 (9.96 MB/s) - ‘RFdiffusion_Ab.pt’ saved [483452922/483452922]

--2025-03-02 16:41:51--  https://files.ipd.uw.edu/pub/RFantibody/ProteinMPNN_v48_noise_0.2.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.135, 128.95.160.134, 2607:4000:406::160:135, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6681301 (6.4M) [application/octet-stream]
Saving to: ‘ProteinMPNN_v48_noise_0.2.pt’

Pr

In [5]:
# Step 6: Start Docker container with GPU support
!docker run --name rfantibody --gpus all -v $PWD:/home --rm -itd rfantibody

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?

Run 'docker run --help' for more information


In [ ]:
# Step 7: Install Python dependencies inside container
!docker exec -it rfantibody bash -c "bash /home/include/setup.sh"

Path /content/RFantibody/include/dgl/dgl-2.4.0+cu118-cp310-cp310-manylinux1_x86_64.whl for dgl does not exist
Installing dependencies from lock file

pyproject.toml changed significantly since poetry.lock was last generated. Run `poetry lock` to fix the lock file.


In [ ]:
# 1. Completely remove the existing environment
!rm -rf .venv/
!rm poetry.lock


rm: cannot remove 'poetry.lock': No such file or directory


In [ ]:

# 2. Force-update pyproject.toml
!sed -i '/dgl = {path =/d' pyproject.toml
!echo 'dgl = {version = "2.1.0", extras = ["cu122"]}' >> pyproject.toml

# 3. Install CUDA 12.2 compatible PyTorch first
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121

# 4. Reinitialize Poetry with clean install
!poetry config virtualenvs.in-project true
!poetry env use python3.10
!poetry install --no-cache

# 5. Verify installation
!poetry run python -c "import dgl; print(f'DGL version: {dgl.__version__}')"
!poetry run python -c "import torch; print(f'PyTorch CUDA: {torch.version.cuda}')"

In [ ]:
%%bash include/download_weights.sh

Weights directory already exists! Skipping download.


In [ ]:
!source .venv/bin/activate


In [ ]:
!pip install numpy

In [ ]:
# Example command (replace `mychothia.pdb` with your file)
!poetry run python /content/RFantibody/scripts/util/chothia2HLT.py \
    -inpdb /content/RFantibody/scripts/examples/example_inputs/h-NbBCII10.pdb \
    -outpdb myHLT.pdb

Path /content/RFantibody/include/dgl/dgl-2.4.0+cu118-cp310-cp310-manylinux1_x86_64.whl for dgl does not exist
Traceback (most recent call last):
  File "/content/RFantibody/scripts/util/chothia2HLT.py", line 12, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'


In [ ]:
# Run RFdiffusion
!poetry run python /content/RFantibody/scripts/rfdiffusion_inference.py \
    --config-name antibody \
    antibody.target_pdb=/content/RFantibody/scripts/examples/example_inputs/rsv_site3.pdb \
    antibody.framework_pdb=/content/RFantibody/scripts/examples/example_inputs/hu-4D5-8_Fv.pdb \
    inference.ckpt_override_path=weights/RFdiffusion_Ab.pt \
    'ppi.hotspot_res=[T305,T456]' \
    'antibody.design_loops=[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]' \
    inference.num_designs=5 \
    inference.output_prefix=outputs/ab_des

Path /content/RFantibody/include/dgl/dgl-2.4.0+cu118-cp310-cp310-manylinux1_x86_64.whl for dgl does not exist
Traceback (most recent call last):
  File "/content/RFantibody/scripts/rfdiffusion_inference.py", line 24, in <module>
    import torch 
ModuleNotFoundError: No module named 'torch'


In [ ]:
%%bash include/download_weights.sh

All weights downloaded successfully!


--2025-03-01 18:55:55--  https://files.ipd.uw.edu/pub/RFantibody/RFdiffusion_Ab.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.135, 128.95.160.134, 2607:4000:406::160:134, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 483452922 (461M) [application/octet-stream]
Saving to: ‘RFdiffusion_Ab.pt’

     0K .......... .......... .......... .......... ..........  0% 1.48M 5m13s
    50K .......... .......... .......... .......... ..........  0%  755K 7m49s
   100K .......... .......... .......... .......... ..........  0%  196M 5m13s
   150K .......... .......... .......... .......... ..........  0% 1.45M 5m14s
   200K .......... .......... .......... .......... ..........  0% 71.6M 4m13s
   250K .......... .......... .......... .......... ..........  0%  106M 3m31s
   300K .......... .......... .......... .......... ..........  0% 1.57M 3m43s
   350K .......... .......... ..........

In [ ]:
!sudo usermod -aG docker $USER

Usage: usermod [options] LOGIN

Options:
  -b, --badnames                allow bad names
  -c, --comment COMMENT         new value of the GECOS field
  -d, --home HOME_DIR           new home directory for the user account
  -e, --expiredate EXPIRE_DATE  set account expiration date to EXPIRE_DATE
  -f, --inactive INACTIVE       set password inactive after expiration
                                to INACTIVE
  -g, --gid GROUP               force use GROUP as new primary group
  -G, --groups GROUPS           new list of supplementary GROUPS
  -a, --append                  append the user to the supplemental GROUPS
                                mentioned by the -G option without removing
                                the user from other groups
  -h, --help                    display this help message and exit
  -l, --login NEW_LOGIN         new value of the login name
  -L, --lock                    lock the user account
  -m, --move-home               move contents of the home direc

In [ ]:
!docker build -t rfantibody .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [ ]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker pull hello-world
docker images
docker run hello-world
kill $(jobs -p)

++ seq 5
+ dockerd -b none --iptables=0 -l warn
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
ERRO[2020-10-08T22:37:51.078238743Z] failed to change OOM score to -500            error="write /proc/2324/oom_score_adj: permission denied"
WARN[2020-10-08T22:37:51.107084831Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2020-10-08T22:37:51.124736559Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2020-10-08T22:37:51.173455629Z] 'overlay2' is not supported over overlayfs    storage-driver=overlay2
ERRO[2020-10-08T22:37:51.178983624Z] AUFS was not found in /proc/filesystems       storage-driver=aufs
ERRO[2020-10-08T22:37:51.182277059Z] 'overlay' is not supported over overlayfs     storage-driver=overlay
WARN[2020-10-08T22:37:51.264753336Z] Running iptables --wait -t nat -L -n failed with message: `iptables v1.6.1: can't initialize iptables table `nat': Permission denied